In [ ]:
from pymediainfo import MediaInfo
import numpy as np

sample_video = '20241125-NOVA_EQUIPE_na_Frmula_1.mp4'

# Extract media information
media_info = MediaInfo.parse(sample_video)

# Initialize variables to store audio track count and metadata
audio_tracks = []
audio_metadata = []

# Iterate through tracks to find audio tracks
for track in media_info.tracks:
    if track.track_type == "Audio":
        audio_tracks.append(track)
        audio_metadata.append(track.to_data())

# Output the number of audio tracks and their metadata
print(f"Number of audio tracks: {len(audio_tracks)}")
print("Audio track metadata:")
for idx, metadata in enumerate(audio_metadata, start=1):
    print(f"Track {idx}: {metadata}")

In [ ]:
for key, value in audio_metadata[0].items():
    print(f"{key}: {value}")

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))


from detector.silence_librosa import SilenceLibrosaDetector